In [1]:
# Setup the library
import pyrogue as pr
import os, sys
import matplotlib.pyplot as plt
import time
import numpy as np
import math

top_level=f'{os.getcwd()}/../../../'
pr.addLibraryPath( f'{top_level}/scripts' )
import setupLibPaths
import epix_hr_single_10k
import ePixViewer.Cameras as cameras

%run ../supportFunctions.ipynb

In [2]:
# Create and start the root class
root = epix_hr_single_10k.RootLCLSIITiming(
    top_level   = top_level,
    sim         = False,
    dev         = '/dev/datadev_0',
    asicVersion = 4
)
root.start()

Simulation mode : False
Start: Started zmqServer on ports 9099-9101
    To start a gui: python -m pyrogue gui --server='localhost:9099'
    To use a virtual client: client = pyrogue.interfaces.VirtualClient(addr='localhost', port=9099)
ERROR:pyrogue.PollQueue:Block::checkTransaction: General Error: Transaction error for block ePixHr10kT.EpixHR.BatcherEventBuilder2.NUM_SLAVES_G with address 0x1a000ff4. Error Non zero status message returned on fpga register bus in hardware: 0x80
Traceback (most recent call last):
  File "/afs/slac.stanford.edu/g/reseng/vol31/anaconda/anaconda3/envs/rogue_v6.1.3/lib/python3.9/site-packages/pyrogue/_PollQueue.py", line 200, in _poll
    pr.checkTransaction(entry.block)
  File "/afs/slac.stanford.edu/g/reseng/vol31/anaconda/anaconda3/envs/rogue_v6.1.3/lib/python3.9/site-packages/pyrogue/_Block.py", line 70, in checkTransaction
    block._checkTransaction()
rogue.GeneralError: Block::checkTransaction: General Error: Transaction error for block ePixHr10kT.Ep

In [ ]:
# Create useful pointers
AxiVersion = root.Core.AxiVersion
print ( '###################################################')
print ( '#             Firmware Version                    #')
print ( '###################################################')
AxiVersion.printStatus()
print ( '###################################################')

# Baseline TEST

In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask
initSequence = 5

In [ ]:
#Initialze the camera
#[routine, asic0, asic1, asic2, asic3]'
root.EpixHR.InitASIC([initSequence,0,0,1,0])
#points to the ASIC device under test
ASIC = root.EpixHR.Hr10kTAsic2
root.EpixHR.BatcherEventBuilder0.Blowoff.set(True)
root.EpixHR.BatcherEventBuilder0.Bypass.set(1)
root.EpixHR.BatcherEventBuilder1.Blowoff.set(False)
root.EpixHR.BatcherEventBuilder1.Bypass.set(1)
root.EpixHR.BatcherEventBuilder2.Blowoff.set(True)

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(10)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)


In [ ]:
#use system with batcher and missing ASICs
root.EpixHR.SspLowSpeedDecoderReg.enable.set(True)
print ("Locked lanes: %x" % root.EpixHR.SspLowSpeedDecoderReg.Locked.get())
#Only ASIC 1 is available so disable the 6 lower lanes
root.EpixHR.PacketRegisters0.DisableLane.set(0x3FF)
root.EpixHR.PacketRegisters1.DisableLane.set(0xFC0)
#using internal triggers, so bypassing timing channel at the batcher 
root.EpixHR.BatcherEventBuilder0.Bypass.set(1)
root.EpixHR.BatcherEventBuilder1.Bypass.set(1)


In [ ]:
print(ASIC.Preamp.get())
print(ASIC.Vld1_b.get())
print(ASIC.Pixel_CB.get())
print(ASIC.Filter_DAC.get())
print(ASIC.VRef_DAC.get())

In [ ]:
print('DHG', ASIC.DHg.get())
print('RefGenB', ASIC.RefGenB.get())
print('RefGenC', ASIC.RefGenC.get())
print('S2D_1_b', ASIC.S2D_1_b.get())
print('shvc_DAC', ASIC.shvc_DAC.get())
print('S2dDacBias', ASIC.S2dDacBias.get())
#after lorenzo optimized the ADC
#https://confluence.slac.stanford.edu/pages/viewpage.action?spaceKey=ppareg&title=V4+-+ADC+tuning
#baseline level should be around 2.5k
#high gain the threshold should be 11k to switch from H/M to low
          #lorenzo#reduced banding#default
RefGenB = 1       #2              #3
RefGenC = 2       #3              #2
S2D_1_b = 0       #0              #0
shvc_DAC = 30     #23             #23
S2dDacBias = 3    #3              #3
DHg = False
print("")
ASIC.DHg.set(DHg)  
ASIC.RefGenB.set(RefGenB)
ASIC.RefGenC.set(RefGenC)
ASIC.S2D_1_b.set(S2D_1_b)
ASIC.shvc_DAC.set(shvc_DAC)
ASIC.S2dDacBias.set(S2dDacBias)
print('DHG', ASIC.DHg.get())
print('RefGenB', ASIC.RefGenB.get())
print('RefGenC', ASIC.RefGenC.get())
print('S2D_1_b', ASIC.S2D_1_b.get())
print('shvc_DAC', ASIC.shvc_DAC.get())
print('S2dDacBias', ASIC.S2dDacBias.get())

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

PreampList = [4,7,6,5,3,2,1,0] #4
Vld1_b = 1 #1
Pixel_CB= 4 #4
Filter_DAC= 38 #33
VRef_DAC = 53 #22

ASIC.ClearMatrix(0)

for Preamp in PreampList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_2V6Supply_clearMatrix_0_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ########################################################################################################################
    if os.path.exists(filename):
       os.remove(filename)
       print("")
       print("The file %s was removed" %(filename))
       print("")
    else:
       print("The file does not exist")
    imgDesc = run_asic_test(4000)
    np.save(filename, imgDesc)
    ########################################################################################################################
    #[noiseMap, darkAvg] = noise_print()

In [ ]:
print('DHG', ASIC.DHg.get())
print('RefGenB', ASIC.RefGenB.get())
print('RefGenC', ASIC.RefGenC.get())
print('S2D_1_b', ASIC.S2D_1_b.get())
print('shvc_DAC', ASIC.shvc_DAC.get())
print('S2dDacBias', ASIC.S2dDacBias.get())



#scatter plot
plt.figure(1,figsize=(12,8),dpi=150)
dataset = imgDesc[:,10,10]
plt.plot(dataset, 'rx', label=f'\Pixel 10, 10') 
plt.title("baseline") 
plt.legend()
plt.show()

plt.figure(2,figsize=(12,8),dpi=150)
plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
plt.yscale('log')
plt.title("histogram") 
plt.legend()
plt.show()


plt.figure(1,figsize=(12,8),dpi=150)
dataset = imgDesc[:,140,75]
plt.plot(dataset, 'bx', label=f'\somelabel') 
plt.title("basekine") 
plt.legend()
plt.show()

plt.figure(2,figsize=(12,8),dpi=150)
plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
plt.yscale('log')
plt.title("histogram") 
plt.legend()
plt.show()


In [ ]:
imgDesc.shape

In [ ]:
noise_print_roi(x=65,y=134,width=40,height=10, imgDesc=imgDesc)

In [ ]:
noise_print_roi(x=10,y=10,width=50,height=50, imgDesc=imgDesc)

In [ ]:
plt.figure(1,figsize=(6,4),dpi=150)
plt.imshow(imgDesc[100,:,0:192], cmap='gray', vmin=5000, vmax=7000, interpolation='none')
plt.colorbar(location='right', anchor=(0, 0.3), shrink=0.7)

In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

PreampList = [4,7,6,5,4,3,2,1,0] #[4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

RefGenB    = 1     #3       #2              #3
RefGenC    = 2     #0       #3              #2
S2D_1_b    = 0     #0       #0              #0
shvc_DAC   = 30    #30      #23             #23
S2dDacBias = 3     #3       #3              #3
DHg        = False #True    #True           #True 

for Preamp in PreampList:
    
    ######################################################################################################################
    # Remove previous dataset
    #filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_clearMatrix_0_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ########################################################################################################################
    imgDesc_raw = read_asic_test(18000)
    imgDesc = np.bitwise_and(imgDesc, 0xFFFC)
    ########################################################################################################################
    #[noiseMap, darkAvg] = noise_print()
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    
    noise_print_roi(x=10,y=10,width=50,height=50,imgDesc=imgDesc)

MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask


In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xfffe
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ########################################################################################################################
    imgDesc = read_asic_test(20000)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    
    noise_print_roi(x=10,y=10,width=50,height=50)

MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask


In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xfffc
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ########################################################################################################################
    imgDesc = read_asic_test(20000)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    
    noise_print_roi(x=10,y=10,width=50,height=50)

MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask


In [ ]:
# baseline test with external DAC
print(currentCam.bitMask)

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)
    
    ASIC.mTest.set(True)

    #sets HSDAC device
    root.EpixHR.HSDac.enable.set(True)
    #reset status
    root.EpixHR.HSDac.DacChannel.set(1) # 0 means NONE, 1 means SE, 2 Diff, 3 both
    root.EpixHR.HSDac.WFEnabled.set(False)
    root.EpixHR.HSDac.waveformSource.set(0)
    root.EpixHR.HSDac.externalUpdateEn.set(False)
    root.EpixHR.HSDac.run.set(False)

    root.EpixHR.HSDac.DacValue.set(32000)

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_externalDAC_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))
    ########################################################################################################################
    if os.path.exists(filename):
       os.remove(filename)
       print("The file %s was removed" %(filename))
    else:
       print("The file does not exist")
    imgDesc = run_asic_test(20000)
    np.save(filename, imgDesc)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    
    noise_print_roi(x=10,y=10,width=50,height=50)

In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xfffe
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_externalDAC_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))
    ########################################################################################################################
    imgDesc = read_asic_test(20000)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    
    noise_print_roi(x=10,y=10,width=50,height=50)

MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask


In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

Preamp = 4 #[4,7,6,5,3,2,1,0] #4
Vld1_b = 1 #1
Pixel_CB= 4 #4
Filter_DAC= 38 #33
VRef_DAC = 53 #22

          #lorenzo#reduced banding#default
RefGenB = 1       #2              #3
RefGenC = 2       #3              #2
S2D_1_b = 0       #0              #0
shvc_DAC = 30     #23             #23
S2dDacBias = 3    #3              #3
DHg = False
print("")
ASIC.DHg.set(DHg)  
ASIC.RefGenB.set(RefGenB)
ASIC.RefGenC.set(RefGenC)
ASIC.S2D_1_b.set(S2D_1_b)
ASIC.shvc_DAC.set(shvc_DAC)
ASIC.S2dDacBias.set(S2dDacBias)
print('DHG', ASIC.DHg.get())
print('RefGenB', ASIC.RefGenB.get())
print('RefGenC', ASIC.RefGenC.get())
print('S2D_1_b', ASIC.S2D_1_b.get())
print('shvc_DAC', ASIC.shvc_DAC.get())
print('S2dDacBias', ASIC.S2dDacBias.get())

ASIC.ClearMatrix(0)

for VRef_DAC in range(0,64,4):
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_2V6Supply_clearMatrix_0_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ########################################################################################################################
    if os.path.exists(filename):
       #os.remove(filename)
       print("")
       print("The file %s was skipped" %(filename))
       print("")
    else:
       print("The file does not exist")
       imgDesc = run_asic_test(4000)
       np.save(filename, imgDesc)
    ########################################################################################################################
    #[noiseMap, darkAvg] = noise_print()

In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

Preamp = 4 #[4,7,6,5,4,3,2,1,0] #[4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

RefGenB    = 1     #3       #2              #3
RefGenC    = 2     #0       #3              #2
S2D_1_b    = 0     #0       #0              #0
shvc_DAC   = 30    #30      #23             #23
S2dDacBias = 3     #3       #3              #3
DHg        = False #True    #True           #True 


imgDescCum = []

for VRef_DAC in range(0,64,4):
    
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_2V6Supply_clearMatrix_0_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ########################################################################################################################
    imgDesc = read_asic_test(4000)
    if (imgDescCum == []):
        imgDescCum = imgDesc
    else:
        imgDescCum = np.concatenate((imgDescCum, imgDesc),0)
    ########################################################################################################################
    #[noiseMap, darkAvg] = noise_print()
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    
    noise_print_roi(x=10,y=10,width=50,height=50,imgDesc=imgDesc)

MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask


In [ ]:
#/u2/ddoering/ASIC_V4_ASIC2_SB/
#np.save("imgDescCum.npy",imgDescCum)
np.save("imgDescCum2V6Supply.npy",imgDescCum)

In [ ]:
imgDescCumLoaded = np.load("imgDescCum2V6Supply.npy")

In [ ]:
imgDescCumLoaded.shape

In [ ]:
y_index = np.arange(0, 26)

print(y_index.shape)

print((y_index / 3).astype(int))


y_index = np.arange(0, imgDescCumLoaded.shape[0])
y_index = (y_index / 2000).astype(int)*4

print(y_index.shape)


In [ ]:
#scatter plot
plt.figure(1,figsize=(6,4),dpi=150)
dataset = imgDescCumLoaded[:,10,10]
plt.plot(y_index, dataset, 'rx', label=(f'Pixel %d,%d' % (10,10)))
plt.axvline(x=40, color = 'g', label = 'suggested settin: VRef_DAC 40')
plt.axhline(y=3500, color = 'g', label = 'suggested setting')
plt.grid(color='b', linestyle='-', linewidth=2, which='both', axis='y')
plt.title("baseline vs VRef_DAC")
# adding axis labels     
plt.xlabel('VRef_DAC') 
plt.ylabel('ePixHR [ADU]')         
plt.legend()
plt.show()



In [ ]:
#scatter plot
plt.figure(1,figsize=(6,4),dpi=150)
dataset = imgDescCumLoaded[:,10,10]
plt.plot(y_index, dataset, 'rx', label=(f'Pixel %d,%d' % (10,10)))
plt.axvline(x=40, color = 'g', label = 'suggested settin: VRef_DAC 40')
plt.axhline(y=3500, color = 'g', label = 'suggested setting')
plt.grid(color='b', linestyle='-', linewidth=2, which='both', axis='y')
plt.title("baseline vs VRef_DAC")
# adding axis labels     
plt.xlabel('VRef_DAC') 
plt.ylabel('ePixHR [ADU]')         
plt.legend()
plt.show()



In [ ]:
dataset1 = dataset.reshape(16, -1)

In [ ]:
dataset1 = dataset.reshape(64, -1)
print(dataset1.shape)
plt.figure(1,figsize=(12,8),dpi=150)
#dataset = imgDescCumLoaded[:,10,10].reshape(4000,-1)
plt.plot(dataset1.transpose(), 'bx', label=(f'Pixel %d,%d' % (10,10)))
plt.title("baseline") 
#plt.legend()
plt.show()

In [ ]:
print(dataset1.shape)
plt.figure(1,figsize=(12,8),dpi=150)
#dataset = imgDescCumLoaded[:,10,10].reshape(4000,-1)
plt.plot(dataset1[10], 'bx', label=(f'Pixel %d,%d' % (10,10)))
plt.title("baseline") 
#plt.legend()
plt.show()

In [ ]:
plt.figure(2,figsize=(12,8),dpi=150)
plt.hist(dataset1.reshape(-1), bins=100, label=f'\somelabel') 
plt.yscale('log')
plt.title("histogram") 
plt.legend()
plt.show()

In [ ]:
imgDesc13b = np.bitwise_and(dataset1[:],0xFFFE)
imgDesc12b = np.bitwise_and(dataset1[:],0xFFFC)
imgDesc11b = np.bitwise_and(dataset1[:],0xFFF8)

In [ ]:
print(dataset1.shape)
plt.figure(figsize=(6,4),dpi=150)
#dataset = imgDescCumLoaded[:,10,10].reshape(4000,-1)
plt.plot(dataset1[40], 'rx', label=(f'Pixel %d,%d' % (10,10)))
plt.plot(imgDesc13b[40], 'bx', label=(f'Pixel %d,%d' % (10,10)))
plt.title("baseline") 
#plt.legend()
plt.show()

plt.figure(figsize=(6,4),dpi=150)
plt.plot(dataset1[40], 'rx', label=(f'Pixel %d,%d' % (10,10)))
plt.plot(imgDesc12b[40], 'gx', label=(f'Pixel %d,%d' % (10,10)))
plt.title("baseline") 
#plt.legend()
plt.show()

plt.figure(figsize=(6,4),dpi=150)
plt.plot(dataset1[40], 'rx', label=(f'Pixel %d,%d' % (10,10)))
plt.plot(imgDesc11b[40], 'cx', label=(f'Pixel %d,%d' % (10,10)))
plt.title("baseline") 
#plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(6,4),dpi=150)
plt.hist(dataset1[40], bins=25, label=f'14b') 
plt.title("histogram") 
plt.legend()
plt.show()
plt.figure(figsize=(6,4),dpi=150)
plt.hist(imgDesc13b[40], bins=25, label=f'13b') 
plt.title("histogram") 
plt.legend()
plt.show()
plt.figure(figsize=(6,4),dpi=150)
plt.hist(imgDesc12b[40], bins=25, label=f'12b') 
plt.title("histogram") 
plt.legend()
plt.show()
plt.figure(figsize=(6,4),dpi=150)
plt.hist(imgDesc11b[40], bins=25, label=f'11b') 
plt.title("histogram") 
plt.legend()
plt.show()

plt.figure(figsize=(6,4),dpi=150)
plt.hist(dataset1[40], bins=25, label=f'14b') 
plt.hist(imgDesc13b[40], bins=25, label=f'13b') 
plt.hist(imgDesc12b[40], bins=25, label=f'12b') 
plt.hist(imgDesc11b[40], bins=25, label=f'11b') 
plt.title("histogram") 
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(6,4),dpi=150)
plt.hist(dataset1.reshape(-1), bins=100, label=f'14b') 
plt.yscale('log')
plt.title("histogram") 
plt.legend()
plt.show()
plt.figure(figsize=(6,4),dpi=150)
plt.hist(imgDesc13b.reshape(-1), bins=100, label=f'13b') 
plt.yscale('log')
plt.title("histogram") 
plt.legend()
plt.show()
plt.figure(figsize=(6,4),dpi=150)
plt.hist(imgDesc12b.reshape(-1), bins=100, label=f'12b') 
plt.yscale('log')
plt.title("histogram") 
plt.legend()
plt.show()
plt.figure(figsize=(6,4),dpi=150)
plt.hist(imgDesc11b.reshape(-1), bins=100, label=f'11b') 
plt.yscale('log')
plt.title("histogram") 
plt.legend()
plt.show()

# Baseline Matrix in RESET TEST

In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

In [ ]:
#Initialze the camera
#[routine, asic0, asic1, asic2, asic3]'
initSequence = 5
root.EpixHR.InitASIC([initSequence,0,0,1,0])
#points to the ASIC device under test
ASIC = root.EpixHR.Hr10kTAsic2
root.EpixHR.BatcherEventBuilder0.Blowoff.set(True)
root.EpixHR.BatcherEventBuilder0.Bypass.set(1)
root.EpixHR.BatcherEventBuilder1.Blowoff.set(False)
root.EpixHR.BatcherEventBuilder1.Bypass.set(1)
root.EpixHR.BatcherEventBuilder2.Blowoff.set(True)

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(10)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)


In [ ]:
#use system with batcher and missing ASICs
root.EpixHR.SspLowSpeedDecoderReg.enable.set(True)
print ("Locked lanes: %x" % root.EpixHR.SspLowSpeedDecoderReg.Locked.get())
#Only ASIC 1 is available so disable the 6 lower lanes
root.EpixHR.PacketRegisters0.DisableLane.set(0x3FF)
root.EpixHR.PacketRegisters1.DisableLane.set(0xFC0)
#using internal triggers, so bypassing timing channel at the batcher 
root.EpixHR.BatcherEventBuilder0.Bypass.set(1)
root.EpixHR.BatcherEventBuilder1.Bypass.set(1)
root.EpixHR.BatcherEventBuilder2.Blowoff.set(True)


In [ ]:
print(ASIC.Preamp.get())
print(ASIC.Vld1_b.get())
print(ASIC.Pixel_CB.get())
print(ASIC.Filter_DAC.get())
print(ASIC.VRef_DAC.get())

In [ ]:
print('DHG', ASIC.DHg.get())
print('RefGenB', ASIC.RefGenB.get())
print('RefGenC', ASIC.RefGenC.get())
print('S2D_1_b', ASIC.S2D_1_b.get())
print('shvc_DAC', ASIC.shvc_DAC.get())
print('S2dDacBias', ASIC.S2dDacBias.get())
DHg = True
RefGenB = 2
RefGenC = 3
S2D_1_b = 0
shvc_DAC = 23
S2dDacBias = 3
print("")
ASIC.DHg.set(DHg)  
ASIC.RefGenB.set(RefGenB)
ASIC.RefGenC.set(RefGenC)
ASIC.S2D_1_b.set(S2D_1_b)
ASIC.shvc_DAC.set(shvc_DAC)
ASIC.S2dDacBias.set(S2dDacBias)
print('DHG', ASIC.DHg.get())
print('RefGenB', ASIC.RefGenB.get())
print('RefGenC', ASIC.RefGenC.get())
print('S2D_1_b', ASIC.S2D_1_b.get())
print('shvc_DAC', ASIC.shvc_DAC.get())
print('S2dDacBias', ASIC.S2dDacBias.get())

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

PreampList = [7,6,5,4,3,2,1,0] #4
Vld1_b = 1 #1
Pixel_CB= 4 #4
Filter_DAC= 38 #33
VRef_DAC = 53 #22

ASIC.ClearMatrix(2)

for Preamp in PreampList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_clearMatrix_2_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ########################################################################################################################
    if os.path.exists(filename):
       os.remove(filename)
       print("The file %s was removed" %(filename))
    else:
       print("The file does not exist")
    imgDesc = run_asic_test(4000)
    np.save(filename, imgDesc)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()

In [ ]:
#scatter plot
plt.figure(1,figsize=(12,8),dpi=150)
dataset = imgDesc[:,10,10]
plt.plot(dataset, 'rx', label=f'\somelabel') 
plt.title("basekine") 
plt.legend()
plt.show()

plt.figure(2,figsize=(12,8),dpi=150)
plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
plt.yscale('log')
plt.title("histogram") 
plt.legend()
plt.show()


plt.figure(1,figsize=(12,8),dpi=150)
dataset = imgDesc[:,140,75]
plt.plot(dataset, 'bx', label=f'\somelabel') 
plt.title("basekine") 
plt.legend()
plt.show()

plt.figure(2,figsize=(12,8),dpi=150)
plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
plt.yscale('log')
plt.title("histogram") 
plt.legend()
plt.show()


In [ ]:
noise_print_roi(x=65,y=134,width=40,height=10)

In [ ]:
noise_print_roi(x=10,y=10,width=50,height=50)

In [ ]:
plt.figure(1,figsize=(6,4),dpi=150)
plt.imshow(imgDesc[100,:,0:192], cmap='gray', vmin=5000, vmax=7000, interpolation='none')
plt.colorbar(location='right', anchor=(0, 0.3), shrink=0.7)

In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ########################################################################################################################
    imgDesc = read_asic_test(20000)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    
    noise_print_roi(x=10,y=10,width=50,height=50)

MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask


In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xfffe
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ########################################################################################################################
    imgDesc = read_asic_test(20000)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    
    noise_print_roi(x=10,y=10,width=50,height=50)

MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask


In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xfffc
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ########################################################################################################################
    imgDesc = read_asic_test(20000)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    
    noise_print_roi(x=10,y=10,width=50,height=50)

MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask


In [ ]:
# baseline test with external DAC
print(currentCam.bitMask)

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)
    
    ASIC.mTest.set(True)

    #sets HSDAC device
    root.EpixHR.HSDac.enable.set(True)
    #reset status
    root.EpixHR.HSDac.DacChannel.set(1) # 0 means NONE, 1 means SE, 2 Diff, 3 both
    root.EpixHR.HSDac.WFEnabled.set(False)
    root.EpixHR.HSDac.waveformSource.set(0)
    root.EpixHR.HSDac.externalUpdateEn.set(False)
    root.EpixHR.HSDac.run.set(False)

    root.EpixHR.HSDac.DacValue.set(32000)

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_externalDAC_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))
    ########################################################################################################################
    if os.path.exists(filename):
       os.remove(filename)
       print("The file %s was removed" %(filename))
    else:
       print("The file does not exist")
    imgDesc = run_asic_test(20000)
    np.save(filename, imgDesc)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    
    noise_print_roi(x=10,y=10,width=50,height=50)

In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xfffe
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

PreampList = [4]
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53

for Preamp in PreampList:
    
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_baseline_externalDAC_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))
    ########################################################################################################################
    imgDesc = read_asic_test(20000)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    
    noise_print_roi(x=10,y=10,width=50,height=50)

MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask


# Pipo Delay test

In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

In [ ]:
#Initialze the camera
#[routine, asic0, asic1, asic2, asic3]'
initSequence = 5
root.EpixHR.InitASIC([initSequence,0,0,1,0])
#points to the ASIC device under test
ASIC = root.EpixHR.Hr10kTAsic2
root.EpixHR.BatcherEventBuilder0.Blowoff.set(True)
root.EpixHR.BatcherEventBuilder0.Bypass.set(1)
root.EpixHR.BatcherEventBuilder1.Blowoff.set(False)
root.EpixHR.BatcherEventBuilder1.Bypass.set(1)
root.EpixHR.BatcherEventBuilder2.Blowoff.set(True)

In [ ]:
ASIC.pipoclk_delay.get()

In [ ]:
ASIC.mTest.set(True)

#sets HSDAC device
root.EpixHR.HSDac.enable.set(True)
#reset status
root.EpixHR.HSDac.DacChannel.set(1) # 0 means NONE, 1 means SE, 2 Diff, 3 both
root.EpixHR.HSDac.WFEnabled.set(False)
root.EpixHR.HSDac.waveformSource.set(0)
root.EpixHR.HSDac.externalUpdateEn.set(False)
root.EpixHR.HSDac.run.set(False)

root.EpixHR.HSDac.DacValue.set(32000)

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)

Preamp = 4
Vld1_b = 1
Pixel_CB= 4
Filter_DAC= 38
VRef_DAC = 53
pipoDelayList = np.arange(15)

for pipoDelay in pipoDelayList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)
    print(pipoDelay)
    ASIC.pipoclk_delay.set(int(pipoDelay))

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_noise_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_pipoDelay_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC, pipoDelay)) 
    if os.path.exists(filename):
       os.remove(filename)
       print("The file %s was removed" %(filename))
    else:
       print("The file does not exist")
    imgDesc = run_asic_test(numFrames=2000,initialDelay=2.0)
    np.save(filename, imgDesc)
    ########################################################################################################################
    [noiseMap, darkAvg] = glitchFinder_print(x=0,y=0,width=144,height=192)

# Baseline TEST all

In [ ]:
MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask

In [ ]:
#Initialze the camera
#[routine, asic0, asic1, asic2, asic3]'
initSequence = 5
root.EpixHR.InitASIC([initSequence,0,0,1,0])
#points to the ASIC device under test
ASIC = root.EpixHR.Hr10kTAsic2
root.EpixHR.BatcherEventBuilder0.Blowoff.set(True)
root.EpixHR.BatcherEventBuilder0.Bypass.set(1)
root.EpixHR.BatcherEventBuilder1.Blowoff.set(False)
root.EpixHR.BatcherEventBuilder1.Bypass.set(1)
root.EpixHR.BatcherEventBuilder2.Blowoff.set(True)

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(10)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)


In [ ]:
#use system with batcher and missing ASICs
root.EpixHR.SspLowSpeedDecoderReg.enable.set(True)
print ("Locked lanes: %x" % root.EpixHR.SspLowSpeedDecoderReg.Locked.get())
#Only ASIC 1 is available so disable the 6 lower lanes
root.EpixHR.PacketRegisters0.DisableLane.set(0x3FF)
root.EpixHR.PacketRegisters1.DisableLane.set(0xFC0)



In [ ]:
print(ASIC.Preamp.get())
print(ASIC.Vld1_b.get())
print(ASIC.Pixel_CB.get())
print(ASIC.Filter_DAC.get())
print(ASIC.VRef_DAC.get())

In [ ]:
print('DHG', ASIC.DHg.get())
print('RefGenB', ASIC.RefGenB.get())
print('RefGenC', ASIC.RefGenC.get())
print('S2D_1_b', ASIC.S2D_1_b.get())
print('shvc_DAC', ASIC.shvc_DAC.get())
print('S2dDacBias', ASIC.S2dDacBias.get())
DHg = True
RefGenB = 2
RefGenC = 3
S2D_1_b = 0
shvc_DAC = 23
S2dDacBias = 3
print("")
ASIC.DHg.set(DHg)  
ASIC.RefGenB.set(RefGenB)
ASIC.RefGenC.set(RefGenC)
ASIC.S2D_1_b.set(S2D_1_b)
ASIC.shvc_DAC.set(shvc_DAC)
ASIC.S2dDacBias.set(S2dDacBias)
print('DHG', ASIC.DHg.get())
print('RefGenB', ASIC.RefGenB.get())
print('RefGenC', ASIC.RefGenC.get())
print('S2D_1_b', ASIC.S2D_1_b.get())
print('shvc_DAC', ASIC.shvc_DAC.get())
print('S2dDacBias', ASIC.S2dDacBias.get())

In [ ]:
def complete_test(filename):
    ########################################################################################################################
    if os.path.exists(filename):
       os.remove(filename)
       print("The file %s was removed" %(filename))
    else:
       print("The file does not exist")
    imgDesc = run_asic_test(4000)
    np.save(filename, imgDesc)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print(0,imgDesc=imgDesc)
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()

    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    #plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()

    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    #plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    
    noise_print_roi(x=65,y=134,width=40,height=10, imgDesc=imgDesc)    
    
    noise_print_roi(x=10,y=10,width=50,height=50, imgDesc=imgDesc)
    
    plt.figure(1,figsize=(6,4),dpi=150)
    plt.imshow(imgDesc[100,:,0:192], cmap='gray', vmin=5000, vmax=7000, interpolation='none')
    plt.colorbar(location='right', anchor=(0, 0.3), shrink=0.7)
    
    ########################################################################################################################
    imgDesc = read_asic_test(20000)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print(0, imgDesc=imgDesc)
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    
    noise_print_roi(x=10,y=10,width=50,height=50,imgDesc=imgDesc)

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

PreampList = [7,6,5,4,3,2,1,0] #4
Vld1_b = 1 #1
Pixel_CB= 4 #4
Filter_DAC= 38 #33
VRef_DAC = 53 #22

ASIC.ClearMatrix(0)

for Preamp in PreampList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_clearMatrix_2_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ######################################################################################################################
    complete_test(filename)
    
    MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
    cameraType            = 'ePixHr10kTBatcher'
    bitMask               = 0xffff
    currentCam = cameras.Camera(cameraType = cameraType)
    currentCam.bitMask = bitMask




In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

Preamp = 4 #4
Vld1_bList = [3,2,1,0] #1
Pixel_CB= 4 #4
Filter_DAC= 38 #33
VRef_DAC = 53 #22

ASIC.ClearMatrix(0)

for Vld1_b in Vld1_bList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_clearMatrix_2_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ######################################################################################################################
    complete_test(filename)
    
    MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
    cameraType            = 'ePixHr10kTBatcher'
    bitMask               = 0xffff
    currentCam = cameras.Camera(cameraType = cameraType)
    currentCam.bitMask = bitMask




In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

Preamp = 4 #4
Vld1_b = 1 #1
Pixel_CBList= [7,6,5,4,3,2,1,0] #4
Filter_DAC= 38 #33
VRef_DAC = 53 #22

ASIC.ClearMatrix(0)

for Pixel_CB in Pixel_CBList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_clearMatrix_2_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ######################################################################################################################
    complete_test(filename)
    
    MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
    cameraType            = 'ePixHr10kTBatcher'
    bitMask               = 0xffff
    currentCam = cameras.Camera(cameraType = cameraType)
    currentCam.bitMask = bitMask




In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

Preamp = 4 #4
Vld1_b = 1 #1
Pixel_CB= 4 #4
Filter_DACList= [53,48,43,38,33,28,23,18] #33
VRef_DAC = 53 #22

ASIC.ClearMatrix(0)

for Filter_DAC in Filter_DACList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_clearMatrix_2_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ######################################################################################################################
    complete_test(filename)
    
    MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
    cameraType            = 'ePixHr10kTBatcher'
    bitMask               = 0xffff
    currentCam = cameras.Camera(cameraType = cameraType)
    currentCam.bitMask = bitMask




In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.2)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.2)

Preamp = 4 #4
Vld1_b = 1 #1
Pixel_CB= 4 #4
Filter_DAC= 38 #33
VRef_DACList = [53,48,43,38,33,28,23,18] #22

ASIC.ClearMatrix(0)

for VRef_DAC in VRef_DACList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_clearMatrix_2_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_DHg_%d_RefGenB_%d_RefGenC_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC,DHg,RefGenB,RefGenC))
    ######################################################################################################################
    complete_test(filename)
    
    MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
    cameraType            = 'ePixHr10kTBatcher'
    bitMask               = 0xffff
    currentCam = cameras.Camera(cameraType = cameraType)
    currentCam.bitMask = bitMask




# 2

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)

Preamp = 7
Vld1_bList = [0, 1, 2, 3]
Pixel_CB= 7
Filter_DAC= 17
VRef_DAC = 58

for Vld1_b in Vld1_bList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_noise_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))    ########################################################################################################################
    imgDesc = run_asic_test()
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)

Preamp = 7
Vld1_b = 0
Pixel_CBList= [7,6,5,4,3,2,1,0]
Filter_DAC= 17
VRef_DAC = 58

for Pixel_CB in Pixel_CBList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_noise_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))    ########################################################################################################################
    imgDesc = run_asic_test()
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()

In [ ]:
#Makes sure triggers are off
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
time.sleep(0.1)
root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
time.sleep(0.1)

Preamp = 7
Vld1_b = 0
Pixel_CB= 7
Filter_DACList=[7, 13, 15, 17, 19, 21, 27, 37, 47, 57]
VRef_DAC = 58

for Filter_DAC in Filter_DACList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_noise_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))
    ########################################################################################################################
    imgDesc = run_asic_test()
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()

In [ ]:
Preamp = 7
Vld1_b = 0
Pixel_CB= 7
Filter_DAC= 17
VRef_DACList =[3, 13, 23, 33, 43, 48, 53, 58, 63]

for VRef_DAC in VRef_DACList:
    setPixel (Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC)

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_noise_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))    ########################################################################################################################
    imgDesc = run_asic_test()
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print()